## Preambule

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_2676\182510434.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Read data

In [20]:
xr_dataread = xr.open_dataset("K:/ECEMF/T5.2/xr_dataread.nc")
xr_nld_raw = xr.open_dataset("K:/ECEMF/T5.2/Allocations/xr_alloc_NLD.nc")
xr_2040 = xr.open_dataset("K:/ECEMF/T5.2/xr_alloc_2040.nc")
rules = ['GF', 'PCC', 'PC', 'AP', 'ECPC', 'GDR']

In [3]:
df = pd.read_excel("X:/user/dekkerm/Data//UNFCCC_Parties_Groups_noeu.xlsx", sheet_name = "Country groups")
countries_iso = np.array(df["Country ISO Code"])
group_eu = countries_iso[np.array(df["EU"]) == 1]
group_g20 = countries_iso[np.array(df["G20"]) == 1]
all_countries_iso = np.load("K:/ECEMF/T5.2/all_countries.npy", allow_pickle=True)

In [4]:
# reds_2040 = np.zeros(shape=(len(all_countries_iso), 6))
# for cty_i, cty in enumerate(all_countries_iso):
#     emis_1990 = xr_dataread.sel(Time=1990).GHG_hist.sel(Region=cty)
#     xr_cty = xr.open_dataset("K:/ECEMF/T5.2/Allocations/xr_alloc_"+cty+".nc").sel(NegEmis=0.5, NonCO2red=0.5, Temperature=1.5, Risk=0.5, Timing='Immediate', Discount_factor=2.0, Historical_startyear=1950)
#     emis_2040 = xr_cty.sel(Time=2040)[rules]
#     reds_2040[cty_i] = -((emis_2040 - emis_1990)/emis_1990).to_array()

In [67]:
reds_2040 = []
xr_cty2 = xr_2040.sel(NegEmis=0.5, Temperature=1.5, Risk=0.5, Timing='Immediate')
reds_2040_medians = np.zeros(shape=(len(all_countries_iso), 6))
for cty_i, cty in enumerate(all_countries_iso):
    emis_1990 = xr_dataread.sel(Time=1990).GHG_hist.sel(Region=cty)
    xr_cty = xr_cty2.sel(Region=cty)
    reds_rules = []
    for rule_i,  rule in enumerate(rules):
        emis_2040 = xr_cty[rules]
        ar = np.array((-((emis_2040 - emis_1990)/emis_1990))[rule]).flatten()
        reds_rules.append(ar)
        reds_2040_medians[cty_i, rule_i] = np.median(ar)
    reds_2040.append(reds_rules)
    print(cty_i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196


In [68]:
emis_2040 = xr_dataread.sel(NegEmis=0.5, NonCO2red=0.5, Temperature=1.5, Risk=0.5, Scenario='SSP2', Timing='Immediate').GHG_globe.sel(Time=2040)
emis_2020 = xr_dataread.sel(NegEmis=0.5, NonCO2red=0.5, Temperature=1.5, Risk=0.5, Scenario='SSP2', Timing='Immediate', Region='EARTH').GHG_hist.sel(Time=2020)
red_w = -float((emis_2040 - emis_2020) / emis_2020)

In [69]:
reds_2040_n = ((reds_2040_medians-red_w) / red_w)
avreds = np.mean(reds_2040_n, axis=1)

In [70]:
# avreds_new = []
# for cty_i, cty in enumerate(all_countries_iso):
#     if cty in group_g20:
#         avreds_new.append(avreds[cty_i])
#     else:
#         avreds_new.append(np.nan)

In [71]:
# reds_2040_g20 = np.nanmean(reds_2040, axis=0)
# reds_2040_p = (reds_2040) / red_w

In [72]:
rulecolors = np.array([(90, 51, 36), (141, 145, 31), (0, 156, 223), (157, 0, 100), (230, 173, 31), (61, 29, 92), ])
rulenames = ['Grandfathering', 'Per capita convergence', 'Immediate per capita', 'Ability to pay', "Equal cumulative\nper capita", 'Greenhouse Development\nRights']
rulecolors_hex = ["#{0:02x}{1:02x}{2:02x}".format(i[0], i[1], i[2]) for i in rulecolors]

## Plot

In [112]:
st = '</b><br><i>(% reduction in 2040 w.r.t. 1990)'
fig = make_subplots(rows=3, cols=5, subplot_titles=('<b>Netherlands'+st, '', '', '', '<b>China'+st, '<b>United States'+st, '', '', '', '<b>India'+st, '<b>Brazil'+st,  '', '', '', '<b>South Africa'+st), 
                                    specs=[[{'type':'box'}, {'type': 'choropleth', 'colspan': 3, 'rowspan': 3}, {}, {}, {'type':'box'}], [{'type':'box'}, {}, {}, {}, {'type':'box'}], [{'type':'box'}, {}, {}, {}, {'type':'box'}]],
                                    horizontal_spacing = 0.03,
                                    vertical_spacing=0.14)
                    # insets=[dict(cell=(1,1), l=0.55, b= 0.43),
                    #         dict(cell=(1,1), l=0., h=0., b=0.,  type='polar')])

fig.add_trace(
    go.Choropleth(
    locations=all_countries_iso,
    z = avreds,
    locationmode = 'ISO-3',
    colorscale = 'RdBu_r',
    zmax = 1.5,
    zmin = -1.5,
    hovertemplate  = all_countries_iso,
    name="",
    marker_line_color='black', 
    marker_line_width=1.5,
    colorbar=dict(len=1, x=1, tickvals = np.arange(-25, 25+0.1, 5), title={"text":"GHG budget<br>2021-2100<br>(Gt CO<sub>2/sub>e)"}, tickfont={'size': 25}),
    showscale=False
), 1, 2)
fig.update_layout(
    geo1 = dict(scope='world', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
)
vals = (avreds*100).round(1)
vals_new = []
for v_i, v in enumerate(vals):
    if all_countries_iso[v_i] in group_g20:
        if all_countries_iso[v_i] in group_eu:
            vn = ''
        else:
            if v > 0:
                vn = all_countries_iso[v_i]+': '+'+'+str(v)+'%'
            else:
                vn = all_countries_iso[v_i]+': '+str(v)+'%'
    else:
        vn=''
    vals_new.append(vn)

# fig.add_trace(go.Scattergeo(
#             locations = all_countries_iso,
#             text = np.array(vals_new),
#             featureidkey="properties.NAME_3",
#             mode = 'text',
#             hoverinfo='skip',
#             #bbox=dict(boxstyle="round", fc='white', ec=None, lw=0, alpha=0.5), 
#             showlegend=False,
#             textfont={'size': 15, 'color': 'black'}), #np.sqrt(np.abs(vals))+14
#             row=1,
#             col=2)

fig.update_geos(visible=False,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                landcolor='lightgrey',
                showcoastlines=False,)
fig.update_geos(visible=True, showcountries=False)

fig.update_layout(height=1000, width=2400)
#fig.update_layout(margin={"t":0,"b":0})

# fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=20),
#                    y=0.98, x=0.3, text='        Polar plots indicate reductions compared to the globe (inner ring)')

# Side plots
for cty_i, cty in enumerate(['NLD', 'USA', 'BRA', 'CHN', 'IND', 'ZAF']):
    row = [1, 2, 3, 1, 2, 3][cty_i]
    col = [1, 1, 1, 5, 5, 5][cty_i]
    cty_full = ['Netherlands', 'United States', 'Brazil',  'China','India', 'South Africa'][cty_i]
    # fig.add_annotation(xref='paper', yref='paper',ax=0, align='center', ay=-0, showarrow=False, font=dict(color='black', size=18),
    #                    y=[1.05, 0.67, 0.29, 1.05, 0.67, 0.29][cty_i],
    #                    x=[0.07, 0.07, 0.07, 0.92, 0.92, 0.92][cty_i],
    #                    text="<b>"+cty_full+"</b>")
    for rule_i, rule in enumerate(rules):
        wh = np.where(all_countries_iso==cty)[0][0]
        val = -100*reds_2040[wh][rule_i]#[0][rule_i]
        #valo = (val-1)/4 +0.5
        #print(cty, rule, valo)
        # if valo>1: valo = 1
        # if valo<0: valo = 0
        fig.add_trace(go.Box(y=val,
                             x=[rule]*len(val),
                             boxpoints=False,
                             showlegend=False,
                             marker_color =rulecolors_hex[rule_i]
            
        ), row, col)

def linshape(col, x0, y0, x1, y1, type='solid'):
    fig.add_shape(type="line", xref='paper', yref='paper',
                    x0=x0,
                    y0=y0,
                    x1=x1,
                    y1=y1,
                    line=dict(color=col, width=1, dash=type))
linshape('black', 0.18, 0.87, 0.508, 0.778)
linshape('black', 0.18, 0.5, 0.34, 0.7)
linshape('black', 0.18, 0.13, 0.42, 0.40)
linshape('black', 0.8, 0.87, 0.65, 0.7)
linshape('black', 0.8, 0.5, 0.63, 0.63)
linshape('black', 0.8, 0.13, 0.54, 0.36)
fig.update_layout(template='plotly_white')
fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False)
fig.update_layout(
    font=dict(size=20)
)
fig.update_annotations(font_size=22)

fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=32),
                   y=1.05, x=0.27, text="<b>Higher or lower reduction targets in 2040 than the globe</b>")

fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=1.01, x=0.28, text='        Positive/red indicates more reductions required than global')
fig.show()

In [12]:
# fig = make_subplots(rows=3, cols=5, #subplot_titles=('Grandfathering (GF)',
#                                                     # 'Per Capita (PC)',
#                                                     # 'Per Capita Convergence (PCC)',
#                                                     # 'Equal Cumulative Per Capita',
#                                                     # 'Ability to pay (AP)',
#                                                     # 'Greenhouse Development Rights'), 
#                                     specs=[[{'type':'polar'}, {'type': 'choropleth', 'colspan': 3, 'rowspan': 3}, {}, {}, {'type':'polar'}], [{'type':'polar'}, {}, {}, {}, {'type':'polar'}], [{'type':'polar'}, {}, {}, {}, {'type':'polar'}]],
#                                     horizontal_spacing = 0.,
#                                     vertical_spacing=0.1)
#                     # insets=[dict(cell=(1,1), l=0.55, b= 0.43),
#                     #         dict(cell=(1,1), l=0., h=0., b=0.,  type='polar')])

# fig.add_trace(
#     go.Choropleth(
#     locations=all_countries_iso,
#     z = avreds,
#     locationmode = 'ISO-3',
#     colorscale = 'RdBu_r',
#     zmax = 1.5,
#     zmin = -1.5,
#     hovertemplate  = all_countries_iso,
#     name="",
#     marker_line_color='black', 
#     marker_line_width=1.5,
#     colorbar=dict(len=1, x=1, tickvals = np.arange(-25, 25+0.1, 5), title={"text":"GHG budget<br>2021-2100<br>(Gt CO<sub>2/sub>e)"}, tickfont={'size': 25}),
#     showscale=False
# ), 1, 2)
# fig.update_layout(
#     geo1 = dict(scope='world', 
#                 showlakes=False,
#                 lakecolor='rgb(255, 255, 255)',
#                 showcoastlines=False,),
# )
# vals = (avreds*100).round(1)
# vals_new = []
# for v_i, v in enumerate(vals):
#     if all_countries_iso[v_i] in group_g20:
#         if all_countries_iso[v_i] in group_eu:
#             vn = ''
#         else:
#             if v > 0:
#                 vn = all_countries_iso[v_i]+': '+'+'+str(v)+'%'
#             else:
#                 vn = all_countries_iso[v_i]+': '+str(v)+'%'
#     else:
#         vn=''
#     vals_new.append(vn)

# # fig.add_trace(go.Scattergeo(
# #             locations = all_countries_iso,
# #             text = np.array(vals_new),
# #             featureidkey="properties.NAME_3",
# #             mode = 'text',
# #             hoverinfo='skip',
# #             #bbox=dict(boxstyle="round", fc='white', ec=None, lw=0, alpha=0.5), 
# #             showlegend=False,
# #             textfont={'size': 15, 'color': 'black'}), #np.sqrt(np.abs(vals))+14
# #             row=1,
# #             col=2)

# fig.update_geos(visible=False,
#                 showlakes=True,
#                 lakecolor='rgb(255, 255, 255)',
#                 projection_type='natural earth',
#                 landcolor='lightgrey',
#                 showcoastlines=False,)
# fig.update_geos(visible=True, showcountries=False)

# fig.update_layout(height=1000, width=2400)
# #fig.update_layout(margin={"t":0,"b":0})

# fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
#                    y=1.05, x=0.3, text="<b>Higher or lower reduction targets in 2040 than the globe</b>")

# fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=20),
#                    y=1.01, x=0.3, text='        Positive/red indicates more reductions required than global')

# fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=20),
#                    y=0.98, x=0.3, text='        Polar plots indicate reductions compared to the globe (inner ring)')

# # Polar plots
# for cty_i, cty in enumerate(['NLD', 'USA', 'BRA', 'CHN', 'IND', 'ZAF']):
#     row = [1, 2, 3, 1, 2, 3][cty_i]
#     col = [1, 1, 1, 5, 5, 5][cty_i]
#     cty_full = ['Netherlands', 'United States', 'Brazil',  'China','India', 'South Africa'][cty_i]
#     fig.add_annotation(xref='paper', yref='paper',ax=0, align='center', ay=-0, showarrow=False, font=dict(color='black', size=18),
#                        y=[1.05, 0.67, 0.29, 1.05, 0.67, 0.29][cty_i],
#                        x=[0.07, 0.07, 0.07, 0.92, 0.92, 0.92][cty_i],
#                        text="<b>"+cty_full+"</b>")
#     for rule_i, rule in enumerate(rules):
#         val = reds_2040_p[all_countries_iso==cty][0][rule_i]
#         #valo = (val-1)/4 +0.5
#         #print(cty, rule, valo)
#         # if valo>1: valo = 1
#         # if valo<0: valo = 0
#         if rule == 'GF': val = 1
#         fig.add_trace(go.Barpolar(
#             r=[val+2],
#             theta=[[i*60 for i in range(6)][rule_i]],
#             width=[60],
#             marker_color=rulecolors_hex[rule_i],
#             marker_line_color="black",
#             marker_line_width=0,
#             showlegend=False,
#             opacity=0.8
#         ), row, col)
#     fig.add_trace(go.Scatterpolar(
#         r=[3]*len(np.linspace(0, 360, 1000)),
#         theta=np.linspace(0, 360, 1000),
#         mode='lines',
#         showlegend=False,
#         line_color='black',
#         line_width=3
#     ), row, col)

# r1 = 0
# r2 = +4
# fig.update_layout(
#     #template=None,
#     polar = dict(
#         radialaxis = dict(range=[r1, r2], showticklabels=False, ticks=''),
#         angularaxis = dict(showticklabels=False, ticks=''),
#         radialaxis_gridcolor="darkgrey",
#     ),
#     polar3 = dict(
#         radialaxis = dict(range=[r1, r2], showticklabels=False, ticks=''),
#         angularaxis = dict(showticklabels=False, ticks=''),
#         radialaxis_gridcolor="darkgrey",
#     ),
#     polar4 = dict(
#         radialaxis = dict(range=[r1, r2], showticklabels=False, ticks=''),
#         angularaxis = dict(showticklabels=False, ticks=''),
#         radialaxis_gridcolor="darkgrey",
#     ),
#     polar5 = dict(
#         radialaxis = dict(range=[r1, r2], showticklabels=False, ticks=''),
#         angularaxis = dict(showticklabels=False, ticks=''),
#         radialaxis_gridcolor="darkgrey",
#     ),
#     polar6 = dict(
#         radialaxis = dict(range=[r1, r2], showticklabels=False, ticks=''),
#         angularaxis = dict(showticklabels=False, ticks=''),
#         radialaxis_gridcolor="darkgrey",
#     ),
#     polar2 = dict(
#         radialaxis = dict(range=[r1, r2], showticklabels=False, ticks=''),
#         angularaxis = dict(showticklabels=False, ticks=''),
#         radialaxis_gridcolor="darkgrey",
#     )
# )

# def linshape(col, x0, y0, x1, y1, type='solid'):
#     fig.add_shape(type="line", xref='paper', yref='paper',
#                     x0=x0,
#                     y0=y0,
#                     x1=x1,
#                     y1=y1,
#                     line=dict(color=col, width=1, dash=type))
# linshape('black', 0.1, 0.87, 0.508, 0.778)
# linshape('black', 0.1, 0.5, 0.34, 0.7)
# linshape('black', 0.1, 0.13, 0.42, 0.40)
# linshape('black', 0.9, 0.87, 0.65, 0.7)
# linshape('black', 0.9, 0.5, 0.63, 0.63)
# linshape('black', 0.9, 0.13, 0.54, 0.36)

# fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False)
# fig.show()

## Save

In [113]:
fig.write_image("X:/user/dekkerm/Projects/EffortSharing/Figures/PBL_report/Fig_outsideeurope.png", scale=3)